## Power drain from 100%

This notebook visualizes and analyses *power drain* while moving. We show that iOS appears to report 100% battery for a while after the test starts, but if the test starts with 80%, we can see the drain immediately. This seems to suggest that iOS has some battery level in "reserve". Because we can't easily account for this, we ensure that we start all our experiments with full phones.

## Import all the dependencies

In [ ]:
import sys
sys.path.append("..")

In [ ]:
# for reading and validating data
import emeval.input.spec_details as eisd
import emeval.input.phone_view as eipv
import emeval.input.eval_view as eiev

In [ ]:
# Visualization helpers
import emeval.viz.phone_view as ezpv
import emeval.viz.eval_view as ezev

In [ ]:
# For plots
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# For maps
import branca.element as bre

In [ ]:
# For displaying dates
import arrow

## Load and validate data

The first issue to note is that we actually have two specs here. The first spec is the checked in `evaluation.spec.sample`, which defines calibration for both stationary and moving instances, and some evaluation trips. However, while starting with the calibration, we noticed some inconsistencies between the power curves. So in order to be more consistent, I defined a second, calibration-only spec `examples/calibration.only.json`, which essentially repeats the calibration experiments multiple times.

After that, I returned to the first set of experiments for the moving calibration and the evaluation.

In [ ]:
DATASTORE_URL = "http://cardshark.cs.berkeley.edu"
AUTHOR_EMAIL = "shankari@eecs.berkeley.edu"
sd_80pct = eisd.SpecDetails(DATASTORE_URL, AUTHOR_EMAIL, "sfba_med_freq_calibration_stationary_only_pilot")
sd_mf = eisd.SpecDetails(DATASTORE_URL, AUTHOR_EMAIL, "sfba_med_freq_calibration_stationary_only")

In [ ]:
pv_80pct = eipv.PhoneView(sd_80pct)

In [ ]:
pv_mf = eipv.PhoneView(sd_mf)

In [ ]:
pv_mf = eipv.PhoneView(sd_mf)

In [ ]:
pv_mf.validate()
pv_80pct.validate()

In [ ]:
ev_80pct = eiev.EvaluationView()
ev_80pct.from_view_multiple_runs(pv_80pct, "")
ev_mf = eiev.EvaluationView()
ev_mf.from_view_multiple_runs(pv_mf, "")

In [ ]:
(ifig, [android_ax, ios_ax]) = plt.subplots(ncols=1, nrows=2, figsize=(16,16))

ezpv.plot_all_power_drain(ios_ax, pv_mf.map()["ios"], "calibration", "stationary")
ezpv.plot_all_power_drain(ios_ax, pv_80pct.map()["ios"], "calibration", "stationary")
ios_ax.legend(loc="upper left", mode="expand", bbox_to_anchor=(0, 3.5, 1.25,0.2), ncol=2, prop={"size": 12})
ezpv.plot_all_power_drain(android_ax, pv_mf.map()["android"], "calibration", "stationary")
ezpv.plot_all_power_drain(android_ax, pv_80pct.map()["android"], "calibration", "stationary")
android_ax.legend(loc="lower left", mode="expand", bbox_to_anchor=(0,-2.75,1.25,0.2), ncol=2, prop={"size": 12})

### Plotting individual curves + predicted offset

We also plot the predicted drain if the 80% line had the same characteristics but was offset to start at zero.
And we can see that the resulting curve ends up parallel but consistently below the other two values, mainly due to the initial zero values.

In [ ]:
[r["trip_id"] for r in pv_80pct.map()["ios"]["ucb-sdb-ios-1"]["calibration_ranges"]]

High accuracy is the first range and balanced accuracy is the second range in the two 80% calibration ranges

In [ ]:
(ifig, ax) = plt.subplots(figsize=(12,3), nrows=0, ncols=0)
ezpv.plot_separate_power_drain(ifig, pv_80pct.map()["ios"], 4, "calibration", "high")
ezpv.plot_separate_power_drain(ifig, pv_mf.map()["ios"], 4, "calibration", "high")
added_axes = ifig.axes
# print(len(added_axes))
for i, (phone_label, phone_detail_map) in enumerate(pv_80pct.map()["ios"].items()):
    # print(phone_label, phone_detail_map.keys())
    battery_df = phone_detail_map["calibration_ranges"][0]["battery_df"]
    start_charge = battery_df.battery_level_pct.iloc[0]
    battery_df["offset_pct"] = battery_df.battery_level_pct + (100 - start_charge)
    battery_df.plot(x="hr", y="offset_pct", ax=added_axes[i], style='--')

In [ ]:
(ifig, ax) = plt.subplots(figsize=(12,3), nrows=0, ncols=0)
ezpv.plot_separate_power_drain(ifig, pv_80pct.map()["ios"], 4, "calibration", "balanced")
ezpv.plot_separate_power_drain(ifig, pv_mf.map()["ios"], 4, "calibration", "balanced")
added_axes = ifig.axes
print(len(added_axes))
for i, (phone_label, phone_detail_map) in enumerate(pv_80pct.map()["ios"].items()):
    # print(phone_label, phone_detail_map.keys())
    battery_df = phone_detail_map["calibration_ranges"][1]["battery_df"] # we can see that the offset value for balanced accuracy is the first one
    start_charge = battery_df.battery_level_pct.iloc[0]
    battery_df["offset_pct"] = battery_df.battery_level_pct + (100 - start_charge)
    battery_df.plot(x="hr", y="offset_pct", ax=added_axes[i], style='--')

### High accuracy diff check first

We can also compute some numbers instead of looking at graphs

In [ ]:
# First, let's check the diff for the high accuracy
battery_80_pct = pv_80pct.map()["ios"]["ucb-sdb-ios-1"]["calibration_ranges"][0]["battery_df"]
print("For the 80 start, high accuracy, start = %d, diff = %d, duration = %d" %
      (battery_80_pct.battery_level_pct.iloc[0], (battery_80_pct.battery_level_pct.iloc[0] - battery_80_pct.battery_level_pct.iloc[-1]), battery_80_pct.hr.max()))

In [ ]:
[r["trip_id"] for r in pv_mf.map()["ios"]["ucb-sdb-ios-1"]["calibration_ranges"]]

In [ ]:
battery_100_pct_list = [r["battery_df"] for r in pv_mf.map()["ios"]["ucb-sdb-ios-1"]["calibration_ranges"][:2]]
for df in battery_100_pct_list:
    print("For the 100 start, high accuracy, start = %d, diff = %d, duration = %d" %
        (df.battery_level_pct.iloc[0], (df.battery_level_pct.iloc[0] - df.battery_level_pct.iloc[-1]), df.hr.iloc[-1]))

In [ ]:
for df in battery_100_pct_list:
    print("For the 100 start, high accuracy, start = %d, diff = %d, duration = %d" %
        (df.battery_level_pct.iloc[0], (df.battery_level_pct.iloc[0] - df.battery_level_pct.iloc[-1]), df.hr.iloc[-1]))

### Medium accuracy check next

In [ ]:
battery_80_pct = pv_80pct.map()["ios"]["ucb-sdb-ios-1"]["calibration_ranges"][1]["battery_df"]
print("For the 80 start, high accuracy, start = %d, diff = %d, duration = %d" %
      (battery_80_pct.battery_level_pct.iloc[0], (battery_80_pct.battery_level_pct.iloc[0] - battery_80_pct.battery_level_pct.iloc[-1]), battery_80_pct.hr.max()))

In [ ]:
battery_100_pct_list = [r["battery_df"] for r in pv_mf.map()["ios"]["ucb-sdb-ios-1"]["calibration_ranges"][2:]]
for df in battery_100_pct_list:
    print("For the 100 start, high accuracy, start = %d, diff = %d, duration = %d" %
        (df.battery_level_pct.iloc[0], (df.battery_level_pct.iloc[0] - df.battery_level_pct.iloc[-1]), df.hr.iloc[-1]))